# Problem Statement

As a Data Scientist for a renowned sit-down restaurant chain with numerous locations across the United States, you are tasked with reducing the frequency of service-related errors. These errors are caused by various factors, some within the control of waiters (such as order mishandling) and others outside of their control (such as kitchen errors).

Your objective is to develop a predictive model to identify waiters who are more prone to making preventable errors. This challenge is unprecedented within the organization, requiring you to establish the data preprocessing steps and conduct exploratory data analysis from scratch. This foundational work will pave the way for creating a robust model aimed at minimizing order errors. In addition to preprocessing the dataset, the manager would like for you share some insights. Some ideas for the insights are data quality issues and statistical properties of potential features for the model.




A junior analyst on your team has pulled 3 datasets for you to use in accomplishing your tasks. Below is a summary for each of the tables.



Table: shifts
*   Description: Table where each row represents a single shift performed by a waiter. Only covers shifts in 2022.
*   Columns
  *   EMPLOYEE_ID: An employee's unique identifier
  *   START_TIME: A datetime that describes the start of an employee's shift
  *   END_TIME: A datetime that describes the end of an employee's shift
  *   DURATION: The duration of the shift, in hours

Table: hr
*   Description: Table that contains information about the waiter's hiring dates and the location that they have been hired at. The analyst told you that waiters can switch locations.
*   Columns
  *   EMPLOYEE_ID: An employee's unique identifier
  *   RESTARAUNT_ID: A restaurant location's unique unique identifier
  *   EARLIEST_HIRE_DATE: A date that describes the waiter's first date of employment with the company
  *   RESTAURANT_HIRE_DATE: A date that describes the waiter's first date of employment with a particular restaurant location

Table: errors
*   Description: Table where each row describes an error that has occured.
*   Columns
  *   EMPLOYEE_ID: An employee's unique identifier
  *   RESTARAUNT_ID: A restaurant location's unique unique identifier
  *   EVENT_TIME: The time the error occured
  *   EVENT_DATE: The date the error occured
  *   ERROR_IS_PREVENTABLE: A binary indicator that is set to 1 if the error was preventable.

# Deliverables


*   Dataset ready to be used for modeling. Call it errors_shift_model. You do not need to save it anywhere external.
*   Explorations and insights in the form of comments, tables, and visualizations.
*   A model that meets the project's objective.



# Guidelines / Helpful Tips


*   The datasets provided have not been validated so they may or may not need to be cleaned.
*   There are a great deal of ambigious details about the project, you are expected to do your best to handle them. In order to do this, you will need to make assumptions and structure logic as you see fit.
*   Please use python as the programming language.
*   Please use pandas as your primary tool for data manipulation.
*   Please use sklearn/statsmodels/xgboost to perform any modeling.
*   Please do **not** use the generative AI functionality in Colab.
*   Performance of the model is only a single component of your assessment.
*   The purpose of this takehome is to see how you...
  * structure your logic and attempt to understand ambigious problems
  * display your thought process
*   Please do not use more than 3-4 hours on this assignment. The Penske Data Science team understands that given the time constraint, you will most likely not be able to create a perfect solution. Do the best you can, given the limitations set upon you.
*   You are free to use the internet



# Notebook Setup

In [90]:
# run this cell to initialize the datasets that have been described above
# DO NOT MODIFY THIS CELL

import pandas as pd

shifts = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQWZCylyYjgQO3MUfQvyB1KppG03P2L2RlXOySSolIYI1Dm3Q0gUYH5N3y5xa1HHSrlvzlQk9HxlQC3/pub?output=csv')
hr = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSMuod6qmnAcvkw7z48d1_XL6GGw8j6_yD5d2Jt7OpJN6Oqd4Fwvu0EPQQD7sITati_0MDrtJy6sUMw/pub?output=csv')
errors = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTntMuVEjUUPzxMUB8y4fR6tfOwfHyXVgtB08oYaM_JfkK3BVzi0DbduRFYhFjFD_XtxhREXZvSHzAX/pub?output=csv')

In [91]:
shifts.head()

,EMPLOYEE_ID,START_TIME,END_TIME,DURATION
0,544477416,7/12/2022 5:38,7/12/2022 16:00,10.366667
1,544606615,8/11/2022 14:58,8/11/2022 23:30,8.533333
2,544613149,2/23/2022 1:00,2/23/2022 11:34,10.566667
3,544679188,12/15/2022 4:30,12/15/2022 15:30,11.000000
4,544657030,9/20/2022 8:45,9/20/2022 18:02,9.283333


In [92]:
hr.head()

,EMPLOYEE_ID,RESTAURANT_ID,EARLIEST_HIRE_DATE,RESTAURANT_HIRE_DATE
0,544444763,411857,11/8/1999,5/31/2017
1,544444763,411857,11/8/1999,5/31/2017
2,544444763,411857,11/8/1999,5/31/2017
3,544444763,411857,11/8/1999,5/31/2017
4,544444763,411857,11/8/1999,5/31/2017


In [93]:
errors.head()

,EMPLOYEE_ID,RESTAURANT_ID,EVENT_TIME,EVENT_DATE,ERROR_IS_PREVENTABLE
0,544669126,480256,4:30:00 PM,12/29/2022,1.0
1,544664185,436156,12:20:00 AM,5/24/2022,1.0
2,544672153,503156,9:48:00 PM,5/22/2022,1.0
3,544480270,241956,7:40:00 PM,5/24/2022,1.0
4,544661341,536156,1:00:00 PM,5/31/2022,1.0


# Candidate Starting Point

Please write all your code after this cell.

# Outline of Efforts

# 1. Analyze Data

## Basic Structure of Tables

In [94]:
def print_dataframe_info(df):
    """
    Print basic information about a dataframe including its size, column names,
    and any columns with missing data.

    Parameters:
    df (pd.DataFrame): The dataframe to analyze.
    """
    print(f"Number of rows: {df.shape[0]}")
    print(f"Number of columns: {df.shape[1]}")
    print(f"Column names: {df.columns.tolist()}")
    
    # Checking for columns with missing data
    missing_data = df.isnull().sum()
    missing_columns = missing_data[missing_data > 0]
    
    if not missing_columns.empty:
        print("Columns with missing data:")
        for column, count in missing_columns.items():
            print(f"{column}: {count} missing entries")
    else:
        print("No missing data in any column.")

In [95]:
print_dataframe_info(hr)

Number of rows: 128351
Number of columns: 4
Column names: ['EMPLOYEE_ID', 'RESTAURANT_ID', 'EARLIEST_HIRE_DATE', 'RESTAURANT_HIRE_DATE']
No missing data in any column.


In [96]:
print_dataframe_info(shifts)

Number of rows: 77086
Number of columns: 4
Column names: ['EMPLOYEE_ID', 'START_TIME', 'END_TIME', 'DURATION']
No missing data in any column.


In [97]:
print_dataframe_info(errors)

Number of rows: 337
Number of columns: 5
Column names: ['EMPLOYEE_ID', 'RESTAURANT_ID', 'EVENT_TIME', 'EVENT_DATE', 'ERROR_IS_PREVENTABLE']
Columns with missing data:
ERROR_IS_PREVENTABLE: 1 missing entries


In [98]:
print(errors[errors.isnull().any(axis=1)])

     EMPLOYEE_ID  RESTAURANT_ID  EVENT_TIME EVENT_DATE  ERROR_IS_PREVENTABLE
134    544667738         548656  2:15:00 PM   2/3/2022                   NaN


In [99]:
errors

,EMPLOYEE_ID,RESTAURANT_ID,EVENT_TIME,EVENT_DATE,ERROR_IS_PREVENTABLE
0,544669126,480256,4:30:00 PM,12/29/2022,1.0
1,544664185,436156,12:20:00 AM,5/24/2022,1.0
2,544672153,503156,9:48:00 PM,5/22/2022,1.0
3,544480270,241956,7:40:00 PM,5/24/2022,1.0
4,544661341,536156,1:00:00 PM,5/31/2022,1.0
...,...,...,...,...,...
332,544659812,575756,8:50:00 PM,12/12/2022,1.0
333,544478027,241956,1:00:00 PM,1/25/2022,1.0
334,544656799,575756,9:30:00 PM,5/20/2022,0.0
335,544665184,536156,6:50:00 AM,3/7/2022,1.0


## Key Column Overlap

In [100]:
# Extract unique employee IDs from each dataset
error_employee_ids = set(errors['EMPLOYEE_ID'].unique())
shift_employee_ids = set(shifts['EMPLOYEE_ID'].unique())
hr_employee_ids = set(hr['EMPLOYEE_ID'].unique())

# Count unique EMPLOYEE_IDs
unique_ids_errors = errors['EMPLOYEE_ID'].nunique()
unique_ids_shifts = shifts['EMPLOYEE_ID'].nunique()
unique_ids_hr = hr['EMPLOYEE_ID'].nunique()

# Print the results
print(f"Number of unique employee IDs in errors df: {unique_ids_errors}")
print(f"Number of unique employee IDs in shifts df: {unique_ids_shifts}")
print(f"Number of unique employee IDs in hr df: {unique_ids_hr}")

# Find the intersection of employee IDs in both datasets
overlapping_ids = error_employee_ids.intersection(shift_employee_ids)

overlapping_ids = overlapping_ids.intersection(hr_employee_ids)

# Count the number of overlapping IDs
number_of_overlapping_ids = len(overlapping_ids)
print(f"Number of unique employee IDs that overlap: {number_of_overlapping_ids}")

Number of unique employee IDs in errors df: 295
Number of unique employee IDs in shifts df: 5763
Number of unique employee IDs in hr df: 31493
Number of unique employee IDs that overlap: 174


In [101]:
# Extract unique employee IDs from each dataset
error_employee_ids = set(errors['RESTAURANT_ID'].unique())
hr_employee_ids = set(hr['RESTAURANT_ID'].unique())

# Count unique EMPLOYEE_IDs
unique_ids_errors = errors['RESTAURANT_ID'].nunique()
unique_ids_hr = hr['RESTAURANT_ID'].nunique()

# Print the results
print(f"Number of unique restaurant IDs in errors df: {unique_ids_errors}")
print(f"Number of unique restaurant IDs in hr df: {unique_ids_hr}")

# Find the intersection of employee IDs in both datasets
overlapping_ids = error_employee_ids.intersection(hr_employee_ids)

# Count the number of overlapping IDs
number_of_overlapping_ids = len(overlapping_ids)
print(f"Number of unique restaurant IDs that overlap: {number_of_overlapping_ids}")

Number of unique restaurant IDs in errors df: 36
Number of unique restaurant IDs in hr df: 242
Number of unique restaurant IDs that overlap: 33


## Determination of Key Column Mins / Maxs

In [102]:
def datetime_min_max(df, column_name):
    """
    Calculate and print min and max for a datetime column in a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the datetime data.
    column_name (str): The name of the column to analyze.

    Returns:
    dict: A dictionary containing the min, max of the datetime column.
    """
    try:
        # Ensure the column is in datetime format
        df[column_name] = pd.to_datetime(df[column_name])
        
        # Calculate min, max, and mean
        min_date = df[column_name].min()
        max_date = df[column_name].max()

        stats = {
            'Minimum': min_date,
            'Maximum': max_date,
        }
        
        # Print the statistics
        for key, value in stats.items():
            print(f"{key}: {value}")

        return stats
    except Exception as e:
        print(f"Error: {e}")

In [103]:
datetime_min_max(shifts, 'START_TIME')

Minimum: 2022-01-01 00:00:00
Maximum: 2022-12-31 18:41:00


{'Minimum': Timestamp('2022-01-01 00:00:00'),
 'Maximum': Timestamp('2022-12-31 18:41:00')}

In [104]:
datetime_min_max(shifts, 'END_TIME')

Minimum: 2022-01-01 10:36:00
Maximum: 2023-01-01 07:40:00


{'Minimum': Timestamp('2022-01-01 10:36:00'),
 'Maximum': Timestamp('2023-01-01 07:40:00')}

In [105]:
datetime_min_max(errors, 'EVENT_DATE')

Minimum: 2022-01-03 00:00:00
Maximum: 2023-01-01 00:00:00


{'Minimum': Timestamp('2022-01-03 00:00:00'),
 'Maximum': Timestamp('2023-01-01 00:00:00')}

In [106]:
datetime_min_max(hr, 'EARLIEST_HIRE_DATE')

Minimum: 1984-02-08 00:00:00
Maximum: 2024-03-08 00:00:00


{'Minimum': Timestamp('1984-02-08 00:00:00'),
 'Maximum': Timestamp('2024-03-08 00:00:00')}

In [107]:
datetime_min_max(hr, 'RESTAURANT_HIRE_DATE')

Minimum: 2005-09-19 00:00:00
Maximum: 2024-03-09 00:00:00


{'Minimum': Timestamp('2005-09-19 00:00:00'),
 'Maximum': Timestamp('2024-03-09 00:00:00')}

In [108]:
def col_stats(df, column_name):
    """
    Calculate and print summary statistics for a datetime column in a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the datetime data.
    column_name (str): The name of the column to analyze.

    Returns:
    dict: A dictionary containing the min, max of the datetime column.
    """
    try:        
        # Calculate min, max, and mean
        min_date = df[column_name].min()
        max_date = df[column_name].max()
        mean = df[column_name].mean()
        std = df[column_name].std()

        stats = {
            'Minimum': min_date,
            'Maximum': max_date,
            'Mean Value': mean,
            'Std Value': std
        }
        
        # Print the statistics
        for key, value in stats.items():
            print(f"{key}: {value}")

        return stats
    except Exception as e:
        print(f"Error: {e}")

In [109]:
col_stats(shifts, 'DURATION')

Minimum: 1.0
Maximum: 18.0
Mean Value: 9.28201337897884
Std Value: 1.9052485455667654


{'Minimum': 1.0,
 'Maximum': 18.0,
 'Mean Value': 9.28201337897884,
 'Std Value': 1.9052485455667654}

## 2. Determine Table Structure for Modeling Data

In [110]:
shifts_hr = pd.merge(shifts, hr, how='left', on='EMPLOYEE_ID').drop_duplicates()
shifts_hr['RESTAURANT_ID'] = shifts_hr['RESTAURANT_ID'].apply(lambda x: int(x) if pd.notna(x) else pd.NA)

shifts_hr.sort_values('EMPLOYEE_ID').head(50)

,EMPLOYEE_ID,START_TIME,END_TIME,DURATION,RESTAURANT_ID,EARLIEST_HIRE_DATE,RESTAURANT_HIRE_DATE
242701,544444763,2022-01-28 05:54:00,2022-01-28 18:00:00,12.100000,526056,1999-11-08,2021-07-05
391530,544444763,2022-06-17 05:54:00,2022-06-17 10:01:00,4.116667,411857,1999-11-08,2017-05-31
391544,544444763,2022-06-17 05:54:00,2022-06-17 10:01:00,4.116667,526056,1999-11-08,2021-07-05
80444,544444763,2022-01-29 05:53:00,2022-01-29 17:00:00,11.116667,411857,1999-11-08,2017-05-31
274910,544444763,2022-08-10 05:54:00,2022-08-10 14:10:00,8.266667,411857,1999-11-08,2017-05-31
274924,544444763,2022-08-10 05:54:00,2022-08-10 14:10:00,8.266667,526056,1999-11-08,2021-07-05
427042,544444763,2022-12-19 05:55:00,2022-12-19 18:00:00,12.083333,411857,1999-11-08,2017-05-31
427056,544444763,2022-12-19 05:55:00,2022-12-19 18:00:00,12.083333,526056,1999-11-08,2021-07-05
158379,544444763,2022-10-19 05:54:00,2022-10-19 14:00:00,8.100000,526056,1999-11-08,2021-07-05
158365,544444763,2022-10-19 05:54:00,2022-10-19 14:00:00,8.100000,411857,1999-11-08,2017-05-31


In [111]:
print_dataframe_info(shifts_hr)

Number of rows: 89772
Number of columns: 7
Column names: ['EMPLOYEE_ID', 'START_TIME', 'END_TIME', 'DURATION', 'RESTAURANT_ID', 'EARLIEST_HIRE_DATE', 'RESTAURANT_HIRE_DATE']
Columns with missing data:
RESTAURANT_ID: 10583 missing entries
EARLIEST_HIRE_DATE: 10583 missing entries
RESTAURANT_HIRE_DATE: 10583 missing entries


In [112]:
import datetime

In [113]:
clean_shifts_hr = shifts_hr.dropna()
clean_shifts_hr['RESTAURANT_HIRE_DATE'] = pd.to_datetime(clean_shifts_hr['RESTAURANT_HIRE_DATE'], errors='coerce')
clean_shifts_hr = clean_shifts_hr[clean_shifts_hr['RESTAURANT_HIRE_DATE'].dt.date < datetime.date(2023, 1, 1)]
print_dataframe_info(clean_shifts_hr)

Number of rows: 73702
Number of columns: 7
Column names: ['EMPLOYEE_ID', 'START_TIME', 'END_TIME', 'DURATION', 'RESTAURANT_ID', 'EARLIEST_HIRE_DATE', 'RESTAURANT_HIRE_DATE']
No missing data in any column.


/tmp/ipykernel_340900/593188611.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_shifts_hr['RESTAURANT_HIRE_DATE'] = pd.to_datetime(clean_shifts_hr['RESTAURANT_HIRE_DATE'], errors='coerce')


In [114]:
clean_shifts_hr.sort_values('EMPLOYEE_ID').head(100)

,EMPLOYEE_ID,START_TIME,END_TIME,DURATION,RESTAURANT_ID,EARLIEST_HIRE_DATE,RESTAURANT_HIRE_DATE
423641,544444763,2022-07-29 05:54:00,2022-07-29 16:05:00,10.183333,526056,1999-11-08,2021-07-05
80458,544444763,2022-01-29 05:53:00,2022-01-29 17:00:00,11.116667,526056,1999-11-08,2021-07-05
274924,544444763,2022-08-10 05:54:00,2022-08-10 14:10:00,8.266667,526056,1999-11-08,2021-07-05
274910,544444763,2022-08-10 05:54:00,2022-08-10 14:10:00,8.266667,411857,1999-11-08,2017-05-31
158365,544444763,2022-10-19 05:54:00,2022-10-19 14:00:00,8.100000,411857,1999-11-08,2017-05-31
...,...,...,...,...,...,...,...
436714,544446245,2022-03-30 05:00:00,2022-03-30 15:00:00,10.000000,313458,1996-07-08,2014-09-15
436706,544446245,2022-03-29 05:00:00,2022-03-29 13:00:00,8.000000,313458,1996-07-08,2014-09-15
436698,544446245,2022-03-28 05:00:00,2022-03-28 13:00:00,8.000000,313458,1996-07-08,2014-09-15
436690,544446245,2022-03-26 05:00:00,2022-03-26 11:00:00,6.000000,313458,1996-07-08,2014-09-15


In [115]:
clean_shifts_hr['START_TIME'] = pd.to_datetime(clean_shifts_hr['START_TIME'])
clean_shifts_hr['RESTAURANT_HIRE_DATE'] = pd.to_datetime(clean_shifts_hr['RESTAURANT_HIRE_DATE'])

# Sort the DataFrame to ensure that the latest hire date for each start time comes last
clean_shifts_hr = clean_shifts_hr.sort_values(by=['EMPLOYEE_ID', 'START_TIME', 'RESTAURANT_HIRE_DATE'], ascending=[True, True, False])

# Drop duplicates keeping the last entry (which, due to our sorting, is the latest hire date that doesn't exceed the start time)
# Assume each employee's hire date must be on or before their shift start date to be valid
filtered_clean_shifts_hr = clean_shifts_hr.drop_duplicates(subset=['EMPLOYEE_ID', 'START_TIME'], keep='last')

# Further filter to ensure the hire date is less than or equal to the start time
filtered_clean_shifts_hr = filtered_clean_shifts_hr[filtered_clean_shifts_hr['RESTAURANT_HIRE_DATE'] <= filtered_clean_shifts_hr['START_TIME']]
filtered_clean_shifts_hr

,EMPLOYEE_ID,START_TIME,END_TIME,DURATION,RESTAURANT_ID,EARLIEST_HIRE_DATE,RESTAURANT_HIRE_DATE
32045,544444763,2022-01-14 00:00:00,2022-01-14 10:00:00,10.000000,411857,1999-11-08,2017-05-31
210449,544444763,2022-01-18 05:54:00,2022-01-18 14:00:00,8.100000,411857,1999-11-08,2017-05-31
242687,544444763,2022-01-28 05:54:00,2022-01-28 18:00:00,12.100000,411857,1999-11-08,2017-05-31
80444,544444763,2022-01-29 05:53:00,2022-01-29 17:00:00,11.116667,411857,1999-11-08,2017-05-31
391530,544444763,2022-06-17 05:54:00,2022-06-17 10:01:00,4.116667,411857,1999-11-08,2017-05-31
...,...,...,...,...,...,...,...
226686,544684017,2022-12-26 16:00:00,2022-12-27 02:00:00,10.000000,519856,2022-12-26,2022-12-26
205278,544684036,2022-12-28 09:25:00,2022-12-28 18:01:00,8.600000,534056,2022-12-25,2022-12-25
281239,544684066,2022-12-28 15:00:00,2022-12-28 19:00:00,4.000000,523356,2022-12-28,2022-12-28
287765,544684068,2022-12-31 13:57:00,2022-12-31 19:28:00,5.516667,483956,2022-12-28,2022-12-28


In [116]:
#Check for duplicates
duplicates = filtered_clean_shifts_hr.duplicated(subset=['EMPLOYEE_ID', 'START_TIME'], keep=False)

# Filter the DataFrame to show only the duplicates
duplicate_rows = filtered_clean_shifts_hr[duplicates]
duplicate_rows

,EMPLOYEE_ID,START_TIME,END_TIME,DURATION,RESTAURANT_ID,EARLIEST_HIRE_DATE,RESTAURANT_HIRE_DATE


In [117]:
errors

,EMPLOYEE_ID,RESTAURANT_ID,EVENT_TIME,EVENT_DATE,ERROR_IS_PREVENTABLE
0,544669126,480256,4:30:00 PM,2022-12-29,1.0
1,544664185,436156,12:20:00 AM,2022-05-24,1.0
2,544672153,503156,9:48:00 PM,2022-05-22,1.0
3,544480270,241956,7:40:00 PM,2022-05-24,1.0
4,544661341,536156,1:00:00 PM,2022-05-31,1.0
...,...,...,...,...,...
332,544659812,575756,8:50:00 PM,2022-12-12,1.0
333,544478027,241956,1:00:00 PM,2022-01-25,1.0
334,544656799,575756,9:30:00 PM,2022-05-20,0.0
335,544665184,536156,6:50:00 AM,2022-03-07,1.0


In [118]:
filtered_clean_shifts_hr['RESTAURANT_ID'] = filtered_clean_shifts_hr['RESTAURANT_ID'].astype(int)
filtered_clean_shifts_hr

,EMPLOYEE_ID,START_TIME,END_TIME,DURATION,RESTAURANT_ID,EARLIEST_HIRE_DATE,RESTAURANT_HIRE_DATE
32045,544444763,2022-01-14 00:00:00,2022-01-14 10:00:00,10.000000,411857,1999-11-08,2017-05-31
210449,544444763,2022-01-18 05:54:00,2022-01-18 14:00:00,8.100000,411857,1999-11-08,2017-05-31
242687,544444763,2022-01-28 05:54:00,2022-01-28 18:00:00,12.100000,411857,1999-11-08,2017-05-31
80444,544444763,2022-01-29 05:53:00,2022-01-29 17:00:00,11.116667,411857,1999-11-08,2017-05-31
391530,544444763,2022-06-17 05:54:00,2022-06-17 10:01:00,4.116667,411857,1999-11-08,2017-05-31
...,...,...,...,...,...,...,...
226686,544684017,2022-12-26 16:00:00,2022-12-27 02:00:00,10.000000,519856,2022-12-26,2022-12-26
205278,544684036,2022-12-28 09:25:00,2022-12-28 18:01:00,8.600000,534056,2022-12-25,2022-12-25
281239,544684066,2022-12-28 15:00:00,2022-12-28 19:00:00,4.000000,523356,2022-12-28,2022-12-28
287765,544684068,2022-12-31 13:57:00,2022-12-31 19:28:00,5.516667,483956,2022-12-28,2022-12-28


In [119]:
clean_errors = errors.dropna()
clean_errors['ERROR_IS_PREVENTABLE'].value_counts()

ERROR_IS_PREVENTABLE
1.0    318
0.0     18
Name: count, dtype: int64

In [120]:
clean_errors = clean_errors[clean_errors['ERROR_IS_PREVENTABLE'] == 1.0]
clean_errors['ERROR_IS_PREVENTABLE'].value_counts()

ERROR_IS_PREVENTABLE
1.0    318
Name: count, dtype: int64

In [121]:
errors_shifts_hr = pd.merge(filtered_clean_shifts_hr, clean_errors, how='left', on=['EMPLOYEE_ID','RESTAURANT_ID'])
errors_shifts_hr['RESTAURANT_ID'] = errors_shifts_hr['RESTAURANT_ID'].apply(lambda x: int(x) if pd.notna(x) else pd.NA)

In [122]:
# Normalize the start and end times to just date for comparison
errors_shifts_hr['START_DATE'] = pd.to_datetime(errors_shifts_hr['START_TIME']).dt.date
errors_shifts_hr['END_DATE'] = pd.to_datetime(errors_shifts_hr['END_TIME']).dt.date
errors_shifts_hr['EVENT_DATE'] = pd.to_datetime(errors_shifts_hr['EVENT_DATE']).dt.date

errors_shifts_hr['START_DATE_TIME'] = pd.to_datetime(errors_shifts_hr['START_TIME']).dt.time
errors_shifts_hr['END_DATE_TIME'] = pd.to_datetime(errors_shifts_hr['END_TIME']).dt.time
errors_shifts_hr['EVENT_TIME'] = pd.to_datetime(errors_shifts_hr['EVENT_TIME'], format='%I:%M:%S %p', errors='coerce').dt.time

errors_shifts_hr['IS_IN_SHIFT'] = errors_shifts_hr.apply(
    lambda x: (pd.notna(x['START_DATE']) and pd.notna(x['EVENT_DATE']) and pd.notna(x['END_DATE']) and pd.notna(x['START_TIME']) and pd.notna(x['EVENT_TIME']) and pd.notna(x['END_TIME'])) and
                                                                                                        (x['START_DATE'] <= x['EVENT_DATE'] <= x['END_DATE']) and (x['START_DATE_TIME'] <= x['EVENT_TIME'] <= x['END_DATE_TIME']),
    axis=1
)

In [123]:
print_dataframe_info(errors_shifts_hr)

Number of rows: 68948
Number of columns: 15
Column names: ['EMPLOYEE_ID', 'START_TIME', 'END_TIME', 'DURATION', 'RESTAURANT_ID', 'EARLIEST_HIRE_DATE', 'RESTAURANT_HIRE_DATE', 'EVENT_TIME', 'EVENT_DATE', 'ERROR_IS_PREVENTABLE', 'START_DATE', 'END_DATE', 'START_DATE_TIME', 'END_DATE_TIME', 'IS_IN_SHIFT']
Columns with missing data:
EVENT_TIME: 43772 missing entries
EVENT_DATE: 43772 missing entries
ERROR_IS_PREVENTABLE: 43772 missing entries


In [124]:
errors_in_shift = errors_shifts_hr[errors_shifts_hr['IS_IN_SHIFT'] == True].sort_values(by='EMPLOYEE_ID', ascending=False)

In [125]:
sample_non_errors = errors_shifts_hr.head(200)

In [126]:
sample = pd.concat([errors_in_shift, sample_non_errors])
sample

,EMPLOYEE_ID,START_TIME,END_TIME,DURATION,RESTAURANT_ID,EARLIEST_HIRE_DATE,RESTAURANT_HIRE_DATE,EVENT_TIME,EVENT_DATE,ERROR_IS_PREVENTABLE,START_DATE,END_DATE,START_DATE_TIME,END_DATE_TIME,IS_IN_SHIFT
68216,544680644,2022-11-14 05:26:00,2022-11-14 15:30:00,10.066667,519856,2022-10-24,2022-10-24,13:00:00,2022-11-14,1.0,2022-11-14,2022-11-14,05:26:00,15:30:00,True
67965,544680047,2022-12-02 07:00:00,2022-12-02 16:31:00,9.516667,521156,2022-10-11,2022-10-11,09:00:00,2022-12-02,1.0,2022-12-02,2022-12-02,07:00:00,16:31:00,True
67900,544680046,2022-11-20 05:59:00,2022-11-20 13:35:00,7.600000,556956,2022-10-17,2022-10-17,07:00:00,2022-11-20,1.0,2022-11-20,2022-11-20,05:59:00,13:35:00,True
67769,544679840,2022-12-27 15:55:00,2022-12-27 23:54:00,7.983333,551456,2022-10-10,2022-10-10,20:05:00,2022-12-27,1.0,2022-12-27,2022-12-27,15:55:00,23:54:00,True
67132,544678714,2022-10-13 05:25:00,2022-10-13 15:27:00,10.033333,556956,2022-08-29,2022-08-29,15:25:00,2022-10-13,1.0,2022-10-13,2022-10-13,05:25:00,15:27:00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,544446245,2022-08-02 04:00:00,2022-08-02 16:00:00,12.000000,313458,1996-07-08,2014-09-15,NaN,NaN,NaN,2022-08-02,2022-08-02,04:00:00,16:00:00,False
196,544446245,2022-08-03 04:00:00,2022-08-03 16:00:00,12.000000,313458,1996-07-08,2014-09-15,NaN,NaN,NaN,2022-08-03,2022-08-03,04:00:00,16:00:00,False
197,544446245,2022-08-04 04:00:00,2022-08-04 16:00:00,12.000000,313458,1996-07-08,2014-09-15,NaN,NaN,NaN,2022-08-04,2022-08-04,04:00:00,16:00:00,False
198,544446245,2022-08-05 04:00:00,2022-08-05 16:00:00,12.000000,313458,1996-07-08,2014-09-15,NaN,NaN,NaN,2022-08-05,2022-08-05,04:00:00,16:00:00,False


In [127]:
#Check for duplicates
duplicates = errors_shifts_hr.duplicated(subset=['EMPLOYEE_ID', 'RESTAURANT_ID','START_TIME'], keep=False)

# Filter the DataFrame to show only the duplicates
duplicate_rows = errors_shifts_hr[duplicates]
duplicate_rows

,EMPLOYEE_ID,START_TIME,END_TIME,DURATION,RESTAURANT_ID,EARLIEST_HIRE_DATE,RESTAURANT_HIRE_DATE,EVENT_TIME,EVENT_DATE,ERROR_IS_PREVENTABLE,START_DATE,END_DATE,START_DATE_TIME,END_DATE_TIME,IS_IN_SHIFT
11566,544598465,2022-01-04 06:55:00,2022-01-04 14:55:00,8.000000,463756,2015-09-24,2017-05-18,14:50:00,2022-09-19,1.0,2022-01-04,2022-01-04,06:55:00,14:55:00,False
11567,544598465,2022-01-04 06:55:00,2022-01-04 14:55:00,8.000000,463756,2015-09-24,2017-05-18,14:29:00,2022-08-29,1.0,2022-01-04,2022-01-04,06:55:00,14:55:00,False
11568,544598465,2022-01-05 06:57:00,2022-01-05 14:59:00,8.033333,463756,2015-09-24,2017-05-18,14:50:00,2022-09-19,1.0,2022-01-05,2022-01-05,06:57:00,14:59:00,False
11569,544598465,2022-01-05 06:57:00,2022-01-05 14:59:00,8.033333,463756,2015-09-24,2017-05-18,14:29:00,2022-08-29,1.0,2022-01-05,2022-01-05,06:57:00,14:59:00,False
11570,544598465,2022-01-06 06:56:00,2022-01-06 14:55:00,7.983333,463756,2015-09-24,2017-05-18,14:50:00,2022-09-19,1.0,2022-01-06,2022-01-06,06:56:00,14:55:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65054,544675931,2022-07-21 06:45:00,2022-07-21 14:45:00,8.000000,436156,2022-07-05,2022-07-05,08:45:00,2022-07-25,1.0,2022-07-21,2022-07-21,06:45:00,14:45:00,False
65055,544675931,2022-07-22 06:45:00,2022-07-22 10:39:00,3.900000,436156,2022-07-05,2022-07-05,10:00:00,2022-07-22,1.0,2022-07-22,2022-07-22,06:45:00,10:39:00,True
65056,544675931,2022-07-22 06:45:00,2022-07-22 10:39:00,3.900000,436156,2022-07-05,2022-07-05,08:45:00,2022-07-25,1.0,2022-07-22,2022-07-22,06:45:00,10:39:00,False
65057,544675931,2022-07-25 06:45:00,2022-07-25 14:32:00,7.783333,436156,2022-07-05,2022-07-05,10:00:00,2022-07-22,1.0,2022-07-25,2022-07-25,06:45:00,14:32:00,False


In [128]:
errors_shifts_hr

,EMPLOYEE_ID,START_TIME,END_TIME,DURATION,RESTAURANT_ID,EARLIEST_HIRE_DATE,RESTAURANT_HIRE_DATE,EVENT_TIME,EVENT_DATE,ERROR_IS_PREVENTABLE,START_DATE,END_DATE,START_DATE_TIME,END_DATE_TIME,IS_IN_SHIFT
0,544444763,2022-01-14 00:00:00,2022-01-14 10:00:00,10.000000,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-01-14,2022-01-14,00:00:00,10:00:00,False
1,544444763,2022-01-18 05:54:00,2022-01-18 14:00:00,8.100000,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-01-18,2022-01-18,05:54:00,14:00:00,False
2,544444763,2022-01-28 05:54:00,2022-01-28 18:00:00,12.100000,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-01-28,2022-01-28,05:54:00,18:00:00,False
3,544444763,2022-01-29 05:53:00,2022-01-29 17:00:00,11.116667,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-01-29,2022-01-29,05:53:00,17:00:00,False
4,544444763,2022-06-17 05:54:00,2022-06-17 10:01:00,4.116667,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-06-17,2022-06-17,05:54:00,10:01:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68943,544684017,2022-12-26 16:00:00,2022-12-27 02:00:00,10.000000,519856,2022-12-26,2022-12-26,NaT,NaT,NaN,2022-12-26,2022-12-27,16:00:00,02:00:00,False
68944,544684036,2022-12-28 09:25:00,2022-12-28 18:01:00,8.600000,534056,2022-12-25,2022-12-25,NaT,NaT,NaN,2022-12-28,2022-12-28,09:25:00,18:01:00,False
68945,544684066,2022-12-28 15:00:00,2022-12-28 19:00:00,4.000000,523356,2022-12-28,2022-12-28,NaT,NaT,NaN,2022-12-28,2022-12-28,15:00:00,19:00:00,False
68946,544684068,2022-12-31 13:57:00,2022-12-31 19:28:00,5.516667,483956,2022-12-28,2022-12-28,NaT,NaT,NaN,2022-12-31,2022-12-31,13:57:00,19:28:00,False


In [129]:
errors_shifts_hr = errors_shifts_hr.drop_duplicates(subset=['EMPLOYEE_ID', 'RESTAURANT_ID','START_TIME','IS_IN_SHIFT'])

In [130]:
#Check for duplicates
duplicates = errors_shifts_hr.duplicated(subset=['EMPLOYEE_ID', 'RESTAURANT_ID','START_TIME'], keep=False)

# Filter the DataFrame to show only the duplicates
duplicate_rows = errors_shifts_hr[duplicates]
duplicate_rows_false = duplicate_rows[duplicate_rows['IS_IN_SHIFT'] == False]

duplicate_rows_false['idx_to_remove'] = duplicate_rows_false.index

# Merge this identifier with the original DataFrame to mark rows to be removed
errors_shifts_hr = errors_shifts_hr.merge(duplicate_rows_false[['idx_to_remove']],
                                         left_index=True, right_on='idx_to_remove', how='left', indicator=True)

# Filter out the rows marked for removal
errors_shifts_hr_non_dup = errors_shifts_hr[errors_shifts_hr['_merge'] == 'left_only']

# Drop the auxiliary columns added for the merge and filtering process
errors_shifts_hr_non_dup = errors_shifts_hr_non_dup.drop(columns=['idx_to_remove', '_merge'])

/tmp/ipykernel_340900/3209760608.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicate_rows_false['idx_to_remove'] = duplicate_rows_false.index


In [131]:
errors_shifts_hr_non_dup.reset_index(inplace=True)
errors_shifts_hr_non_dup.drop(columns=['index'], inplace=True)
errors_shifts_hr_non_dup

,EMPLOYEE_ID,START_TIME,END_TIME,DURATION,RESTAURANT_ID,EARLIEST_HIRE_DATE,RESTAURANT_HIRE_DATE,EVENT_TIME,EVENT_DATE,ERROR_IS_PREVENTABLE,START_DATE,END_DATE,START_DATE_TIME,END_DATE_TIME,IS_IN_SHIFT
0,544444763,2022-01-14 00:00:00,2022-01-14 10:00:00,10.000000,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-01-14,2022-01-14,00:00:00,10:00:00,False
1,544444763,2022-01-18 05:54:00,2022-01-18 14:00:00,8.100000,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-01-18,2022-01-18,05:54:00,14:00:00,False
2,544444763,2022-01-28 05:54:00,2022-01-28 18:00:00,12.100000,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-01-28,2022-01-28,05:54:00,18:00:00,False
3,544444763,2022-01-29 05:53:00,2022-01-29 17:00:00,11.116667,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-01-29,2022-01-29,05:53:00,17:00:00,False
4,544444763,2022-06-17 05:54:00,2022-06-17 10:01:00,4.116667,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-06-17,2022-06-17,05:54:00,10:01:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66417,544684017,2022-12-26 16:00:00,2022-12-27 02:00:00,10.000000,519856,2022-12-26,2022-12-26,NaT,NaT,NaN,2022-12-26,2022-12-27,16:00:00,02:00:00,False
66418,544684036,2022-12-28 09:25:00,2022-12-28 18:01:00,8.600000,534056,2022-12-25,2022-12-25,NaT,NaT,NaN,2022-12-28,2022-12-28,09:25:00,18:01:00,False
66419,544684066,2022-12-28 15:00:00,2022-12-28 19:00:00,4.000000,523356,2022-12-28,2022-12-28,NaT,NaT,NaN,2022-12-28,2022-12-28,15:00:00,19:00:00,False
66420,544684068,2022-12-31 13:57:00,2022-12-31 19:28:00,5.516667,483956,2022-12-28,2022-12-28,NaT,NaT,NaN,2022-12-31,2022-12-31,13:57:00,19:28:00,False


In [132]:
#Check for duplicates
duplicates = errors_shifts_hr_non_dup.duplicated(subset=['EMPLOYEE_ID', 'RESTAURANT_ID','START_TIME'], keep=False)

# Filter the DataFrame to show only the duplicates
duplicate_rows = errors_shifts_hr_non_dup[duplicates]
duplicate_rows

,EMPLOYEE_ID,START_TIME,END_TIME,DURATION,RESTAURANT_ID,EARLIEST_HIRE_DATE,RESTAURANT_HIRE_DATE,EVENT_TIME,EVENT_DATE,ERROR_IS_PREVENTABLE,START_DATE,END_DATE,START_DATE_TIME,END_DATE_TIME,IS_IN_SHIFT


In [133]:
errors_shifts_hr_non_dup

,EMPLOYEE_ID,START_TIME,END_TIME,DURATION,RESTAURANT_ID,EARLIEST_HIRE_DATE,RESTAURANT_HIRE_DATE,EVENT_TIME,EVENT_DATE,ERROR_IS_PREVENTABLE,START_DATE,END_DATE,START_DATE_TIME,END_DATE_TIME,IS_IN_SHIFT
0,544444763,2022-01-14 00:00:00,2022-01-14 10:00:00,10.000000,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-01-14,2022-01-14,00:00:00,10:00:00,False
1,544444763,2022-01-18 05:54:00,2022-01-18 14:00:00,8.100000,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-01-18,2022-01-18,05:54:00,14:00:00,False
2,544444763,2022-01-28 05:54:00,2022-01-28 18:00:00,12.100000,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-01-28,2022-01-28,05:54:00,18:00:00,False
3,544444763,2022-01-29 05:53:00,2022-01-29 17:00:00,11.116667,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-01-29,2022-01-29,05:53:00,17:00:00,False
4,544444763,2022-06-17 05:54:00,2022-06-17 10:01:00,4.116667,411857,1999-11-08,2017-05-31,NaT,NaT,NaN,2022-06-17,2022-06-17,05:54:00,10:01:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66417,544684017,2022-12-26 16:00:00,2022-12-27 02:00:00,10.000000,519856,2022-12-26,2022-12-26,NaT,NaT,NaN,2022-12-26,2022-12-27,16:00:00,02:00:00,False
66418,544684036,2022-12-28 09:25:00,2022-12-28 18:01:00,8.600000,534056,2022-12-25,2022-12-25,NaT,NaT,NaN,2022-12-28,2022-12-28,09:25:00,18:01:00,False
66419,544684066,2022-12-28 15:00:00,2022-12-28 19:00:00,4.000000,523356,2022-12-28,2022-12-28,NaT,NaT,NaN,2022-12-28,2022-12-28,15:00:00,19:00:00,False
66420,544684068,2022-12-31 13:57:00,2022-12-31 19:28:00,5.516667,483956,2022-12-28,2022-12-28,NaT,NaT,NaN,2022-12-31,2022-12-31,13:57:00,19:28:00,False


## 3. Create Feature Columns

In [134]:
df = errors_shifts_hr_non_dup.copy(deep=True)

df['StartTime'] = pd.to_datetime(df['START_TIME'])

# Create MultiIndex
df.set_index(['EMPLOYEE_ID', 'RESTAURANT_ID', 'StartTime'], inplace=True)

# Extract hour from 'StartTime'
df['Shift Hour Start'] = df.index.get_level_values('StartTime').hour

# Calculate TotalWorkHoursToDate for each EmployeeID
df['TotalWorkHoursToDate'] = df.groupby(level='EMPLOYEE_ID')['DURATION'].cumsum()

# Calculate RestaurantWorkHoursToDate for each EmployeeID within the same RestaurantID
df['RestaurantWorkHoursToDate'] = df.groupby(['EMPLOYEE_ID', 'RESTAURANT_ID'])['DURATION'].cumsum()
 
# Calculate rolling sums for the specified days
df['TotalWorkHoursInPrevious7Days'] = df.groupby(['EMPLOYEE_ID','RESTAURANT_ID']).rolling(f'7D', on='START_TIME')['DURATION'].sum()
df['TotalWorkHoursInPrevious3Days'] = df.groupby(['EMPLOYEE_ID','RESTAURANT_ID']).rolling(f'3D', on='START_TIME')['DURATION'].sum()

# Sort by 'StartTime' within each group
df.sort_index(level=['EMPLOYEE_ID', 'RESTAURANT_ID', 'StartTime'], inplace=True)

# Calculate cumulative errors
df['EmployeeErrorsToDate'] = df.groupby(level='EMPLOYEE_ID')['IS_IN_SHIFT'].cumsum()
df['RestaurantTotalErrorsToDate'] = df.groupby(['EMPLOYEE_ID', 'RESTAURANT_ID'])['IS_IN_SHIFT'].cumsum()

# Generate lagged features for cumulative errors with lag of 1
# For rows without a previous value, fill with 0
df['EmployeeErrorsToDateLag1'] = df.groupby(level='EMPLOYEE_ID')['EmployeeErrorsToDate'].shift(1).fillna(0)
df['RestaurantTotalErrorsToDateLag1'] = df.groupby(['EMPLOYEE_ID', 'RESTAURANT_ID'])['RestaurantTotalErrorsToDate'].shift(1).fillna(0)


model_df = df.loc[:,['DURATION','Shift Hour Start', 'TotalWorkHoursToDate', 'RestaurantWorkHoursToDate', 'TotalWorkHoursInPrevious7Days', 'TotalWorkHoursInPrevious3Days','EmployeeErrorsToDateLag1', 'RestaurantTotalErrorsToDateLag1', 'IS_IN_SHIFT']]

# 'TotalWorkHoursInPrevious7Days', 'TotalWorkHoursInPrevious3Days',

In [135]:
model_df

DURATION  Shift Hour Start  \
EMPLOYEE_ID RESTAURANT_ID StartTime                                          
544444763   411857        2022-01-14 00:00:00  10.000000                 0   
                          2022-01-18 05:54:00   8.100000                 5   
                          2022-01-28 05:54:00  12.100000                 5   
                          2022-01-29 05:53:00  11.116667                 5   
                          2022-06-17 05:54:00   4.116667                 5   
...                                                  ...               ...   
544684017   519856        2022-12-26 16:00:00  10.000000                16   
544684036   534056        2022-12-28 09:25:00   8.600000                 9   
544684066   523356        2022-12-28 15:00:00   4.000000                15   
544684068   483956        2022-12-31 13:57:00   5.516667                13   
544684122   534056        2022-12-28 12:06:00   7.683333                12   

                                               TotalWorkHoursToDate  \
EMPLOYEE_ID RESTAURANT_ID StartTime                                   
544444763   411857        2022-01-14 00:00:00             10.000000   
                          2022-01-18 05:54:00             18.100000   
                          2022-01-28 05:54:00             30.200000   
                          2022-01-29 05:53:00             41.316667   
                          2022-06-17 05:54:00             45.433333   
...                                                             ...   
544684017   519856        2022-12-26 16:00:00             10.000000   
544684036   534056        2022-12-28 09:25:00              8.600000   
544684066   523356        2022-12-28 15:00:00              4.000000   
544684068   483956        2022-12-31 13:57:00              5.516667   
544684122   534056        2022-12-28 12:06:00              7.683333   

                                               RestaurantWorkHoursToDate  \
EMPLOYEE_ID RESTAURANT_ID StartTime                                        
544444763   411857        2022-01-14 00:00:00                  10.000000   
                          2022-01-18 05:54:00                  18.100000   
                          2022-01-28 05:54:00                  30.200000   
                          2022-01-29 05:53:00                  41.316667   
                          2022-06-17 05:54:00                  45.433333   
...                                                                  ...   
544684017   519856        2022-12-26 16:00:00                  10.000000   
544684036   534056        2022-12-28 09:25:00                   8.600000   
544684066   523356        2022-12-28 15:00:00                   4.000000   
544684068   483956        2022-12-31 13:57:00                   5.516667   
544684122   534056        2022-12-28 12:06:00                   7.683333   

                                               TotalWorkHoursInPrevious7Days  \
EMPLOYEE_ID RESTAURANT_ID StartTime                                            
544444763   411857        2022-01-14 00:00:00                      10.000000   
                          2022-01-18 05:54:00                      18.100000   
                          2022-01-28 05:54:00                      12.100000   
                          2022-01-29 05:53:00                      23.216667   
                          2022-06-17 05:54:00                       4.116667   
...                                                                      ...   
544684017   519856        2022-12-26 16:00:00                      10.000000   
544684036   534056        2022-12-28 09:25:00                       8.600000   
544684066   523356        2022-12-28 15:00:00                       4.000000   
544684068   483956        2022-12-31 13:57:00                       5.516667   
544684122   534056        2022-12-28 12:06:00                       7.683333   

                                               TotalWorkHoursInPrevious3Days  \

In [136]:
model_df[model_df['IS_IN_SHIFT'] == True]

,,,DURATION,Shift Hour Start,TotalWorkHoursToDate,RestaurantWorkHoursToDate,TotalWorkHoursInPrevious7Days,TotalWorkHoursInPrevious3Days,EmployeeErrorsToDateLag1,RestaurantTotalErrorsToDateLag1,IS_IN_SHIFT
EMPLOYEE_ID,RESTAURANT_ID,StartTime,,,,,,,,,
544465441,422556,2022-12-29 05:43:00,7.533333,5,1581.350000,1581.350000,16.250000,7.533333,0.0,0.0,True
544477404,241956,2022-03-06 06:55:00,8.583333,6,381.850000,381.850000,55.383333,25.700000,0.0,0.0,True
544477431,241956,2022-06-06 06:52:00,8.650000,6,865.050000,865.050000,51.816667,8.650000,0.0,0.0,True
544478027,241956,2022-01-25 08:00:00,7.983333,8,120.716667,120.716667,41.100000,16.000000,0.0,0.0,True
544479576,247156,2022-09-16 06:24:00,8.633333,6,1399.200000,1399.200000,52.033333,25.883333,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...
544678714,556956,2022-10-13 05:25:00,10.033333,5,273.533333,273.533333,40.116667,30.100000,0.0,0.0,True
544679840,551456,2022-12-27 15:55:00,7.983333,15,444.533333,444.533333,31.983333,16.250000,0.0,0.0,True
544680046,556956,2022-11-20 05:59:00,7.600000,5,218.166667,218.166667,50.666667,17.600000,0.0,0.0,True


# 4. Modeling

## Hypothesis Testing (& Subsetting)

In [137]:
import numpy as np

In [138]:
model_df['Class'] = np.where(model_df['IS_IN_SHIFT'], 'Preventable Error', 'No Preventable Error')

In [139]:
model_df.head(100)

DURATION  Shift Hour Start  \
EMPLOYEE_ID RESTAURANT_ID StartTime                                          
544444763   411857        2022-01-14 00:00:00  10.000000                 0   
                          2022-01-18 05:54:00   8.100000                 5   
                          2022-01-28 05:54:00  12.100000                 5   
                          2022-01-29 05:53:00  11.116667                 5   
                          2022-06-17 05:54:00   4.116667                 5   
...                                                  ...               ...   
544446245   313458        2022-02-24 06:00:00  11.000000                 6   
                          2022-02-25 06:00:00  11.000000                 6   
                          2022-02-26 06:00:00   8.000000                 6   
                          2022-03-02 06:55:00   8.050000                 6   
                          2022-03-03 06:57:00   8.016667                 6   

                                               TotalWorkHoursToDate  \
EMPLOYEE_ID RESTAURANT_ID StartTime                                   
544444763   411857        2022-01-14 00:00:00             10.000000   
                          2022-01-18 05:54:00             18.100000   
                          2022-01-28 05:54:00             30.200000   
                          2022-01-29 05:53:00             41.316667   
                          2022-06-17 05:54:00             45.433333   
...                                                             ...   
544446245   313458        2022-02-24 06:00:00            455.250000   
                          2022-02-25 06:00:00            466.250000   
                          2022-02-26 06:00:00            474.250000   
                          2022-03-02 06:55:00            482.300000   
                          2022-03-03 06:57:00            490.316667   

                                               RestaurantWorkHoursToDate  \
EMPLOYEE_ID RESTAURANT_ID StartTime                                        
544444763   411857        2022-01-14 00:00:00                  10.000000   
                          2022-01-18 05:54:00                  18.100000   
                          2022-01-28 05:54:00                  30.200000   
                          2022-01-29 05:53:00                  41.316667   
                          2022-06-17 05:54:00                  45.433333   
...                                                                  ...   
544446245   313458        2022-02-24 06:00:00                 455.250000   
                          2022-02-25 06:00:00                 466.250000   
                          2022-02-26 06:00:00                 474.250000   
                          2022-03-02 06:55:00                 482.300000   
                          2022-03-03 06:57:00                 490.316667   

                                               TotalWorkHoursInPrevious7Days  \
EMPLOYEE_ID RESTAURANT_ID StartTime                                            
544444763   411857        2022-01-14 00:00:00                      10.000000   
                          2022-01-18 05:54:00                      18.100000   
                          2022-01-28 05:54:00                      12.100000   
                          2022-01-29 05:53:00                      23.216667   
                          2022-06-17 05:54:00                       4.116667   
...                                                                      ...   
544446245   313458        2022-02-24 06:00:00                      55.000000   
                          2022-02-25 06:00:00                      55.000000   
                          2022-02-26 06:00:00                      57.000000   
                          2022-03-02 06:55:00                      38.050000   
                          2022-03-03 06:57:00                      35.066667   

                                               TotalWorkHoursInPrevious3Days  \

In [140]:
output_df = model_df.reset_index()
output_df = output_df.drop(columns=['EMPLOYEE_ID','RESTAURANT_ID','StartTime','IS_IN_SHIFT'])
output_df.head(100)

,DURATION,Shift Hour Start,TotalWorkHoursToDate,RestaurantWorkHoursToDate,TotalWorkHoursInPrevious7Days,TotalWorkHoursInPrevious3Days,EmployeeErrorsToDateLag1,RestaurantTotalErrorsToDateLag1,Class
0,10.000000,0,10.000000,10.000000,10.000000,10.000000,0.0,0.0,No Preventable Error
1,8.100000,5,18.100000,18.100000,18.100000,8.100000,0.0,0.0,No Preventable Error
2,12.100000,5,30.200000,30.200000,12.100000,12.100000,0.0,0.0,No Preventable Error
3,11.116667,5,41.316667,41.316667,23.216667,23.216667,0.0,0.0,No Preventable Error
4,4.116667,5,45.433333,45.433333,4.116667,4.116667,0.0,0.0,No Preventable Error
...,...,...,...,...,...,...,...,...,...
95,11.000000,6,455.250000,455.250000,55.000000,27.000000,0.0,0.0,No Preventable Error
96,11.000000,6,466.250000,466.250000,55.000000,33.000000,0.0,0.0,No Preventable Error
97,8.000000,6,474.250000,474.250000,57.000000,30.000000,0.0,0.0,No Preventable Error
98,8.050000,6,482.300000,482.300000,38.050000,8.050000,0.0,0.0,No Preventable Error


In [141]:
import statsmodels.api as sm
from statsmodels.stats.weightstats import ttest_ind

In [142]:
# Identify numeric columns (excluding the 'Class' column if it's not numeric)
numeric_columns = output_df.select_dtypes(include=[np.number]).columns

# Set the significance level
alpha = 0.05

# Loop through each numeric column and perform t-tests
for column in numeric_columns:
    # Split the data into two groups based on the 'Class' column
    group1 = output_df[output_df['Class'] == 'Preventable Error'][column].dropna()
    group2 = output_df[output_df['Class'] == 'No Preventable Error'][column].dropna()

    # Perform t-test
    t_stat, p_value, _ = ttest_ind(group1, group2)

    # Print the results
    print(f"Results for {column}:")
    print(f"  t-statistic: {t_stat}, p-value: {p_value}")
    
    # Interpretation of the hypothesis test
    if p_value < alpha:
        print("  Reject the null hypothesis: there is a significant difference between the groups.")
    else:
        print("  Do not reject the null hypothesis: there is no significant difference between the groups.")
    print("\n")  # New line for better separation between results

Results for DURATION:
  t-statistic: -1.5894372624444586, p-value: 0.11196646709284697
  Do not reject the null hypothesis: there is no significant difference between the groups.


Results for Shift Hour Start:
  t-statistic: -4.114964228838617, p-value: 3.877059601621461e-05
  Reject the null hypothesis: there is a significant difference between the groups.


Results for TotalWorkHoursToDate:
  t-statistic: 6.086727131551202, p-value: 1.1587076272417464e-09
  Reject the null hypothesis: there is a significant difference between the groups.


Results for RestaurantWorkHoursToDate:
  t-statistic: 6.086727131551202, p-value: 1.1587076272417464e-09
  Reject the null hypothesis: there is a significant difference between the groups.


Results for TotalWorkHoursInPrevious7Days:
  t-statistic: 11.649240388393197, p-value: 2.483590411438412e-31
  Reject the null hypothesis: there is a significant difference between the groups.


Results for TotalWorkHoursInPrevious3Days:
  t-statistic: 8.48396

In [143]:
# Identify numeric columns (excluding the 'Class' column if it's not numeric)
numeric_columns = output_df.select_dtypes(include=[np.number]).columns

# Set the significance level
alpha = 0.05

# Dictionary to hold dataframes for each significant column
output_df_subsets = []

# Loop through each numeric column and perform t-tests
for column in numeric_columns:
    # Split the data into two groups based on the 'Class' column
    group1 = output_df[output_df['Class'] == 'Preventable Error'][column].dropna()
    group2 = output_df[output_df['Class'] == 'No Preventable Error'][column].dropna()

    # Perform t-test
    t_stat, p_value, _ = ttest_ind(group1, group2)

    # Check if the null hypothesis is rejected
    if p_value < alpha:
        # Calculate the 10% and 90% quantiles for the 'Preventable Error' group
        # Experimented with cutoffs to improve model performance
        lower_quantile = group1.quantile(0.40)
        upper_quantile = group1.quantile(0.60)
        
        # Create a subset of the original dataframe within these quantiles for this column
        subset = output_df[(output_df[column] >= lower_quantile) & (output_df[column] <= upper_quantile)]
        
        # Append the subset dataframe to the list
        output_df_subsets.append(subset)

# Combine all subsets into a single dataframe based on the union of each numerical predictor's subset
if output_df_subsets:
    final_output_df = pd.concat(output_df_subsets).drop_duplicates()
else:
    final_output_df = pd.DataFrame()

final_output_df

,DURATION,Shift Hour Start,TotalWorkHoursToDate,RestaurantWorkHoursToDate,TotalWorkHoursInPrevious7Days,TotalWorkHoursInPrevious3Days,EmployeeErrorsToDateLag1,RestaurantTotalErrorsToDateLag1,Class
14,11.000000,6,22.000000,22.000000,11.000000,11.000000,0.0,0.0,No Preventable Error
17,8.066667,6,50.066667,50.066667,8.066667,8.066667,0.0,0.0,No Preventable Error
18,6.016667,6,56.083333,56.083333,6.016667,6.016667,0.0,0.0,No Preventable Error
20,8.033333,6,72.166667,72.166667,8.033333,8.033333,0.0,0.0,No Preventable Error
21,8.000000,6,80.166667,80.166667,8.000000,8.000000,0.0,0.0,No Preventable Error
...,...,...,...,...,...,...,...,...,...
66172,8.550000,15,304.950000,304.950000,42.783333,25.616667,0.0,0.0,No Preventable Error
66175,5.916667,15,328.016667,328.016667,23.066667,23.066667,0.0,0.0,No Preventable Error
66295,11.533333,4,31.350000,31.350000,21.266667,21.266667,0.0,0.0,No Preventable Error
66317,11.700000,4,33.983333,33.983333,22.916667,22.916667,0.0,0.0,No Preventable Error


In [144]:
final_output_df[final_output_df['Class'] == 'Preventable Error']

,DURATION,Shift Hour Start,TotalWorkHoursToDate,RestaurantWorkHoursToDate,TotalWorkHoursInPrevious7Days,TotalWorkHoursInPrevious3Days,EmployeeErrorsToDateLag1,RestaurantTotalErrorsToDateLag1,Class
1496,8.583333,6,381.850000,381.850000,55.383333,25.700000,0.0,0.0,Preventable Error
1873,8.650000,6,865.050000,865.050000,51.816667,8.650000,0.0,0.0,Preventable Error
3728,8.633333,6,1399.200000,1399.200000,52.033333,25.883333,0.0,0.0,Preventable Error
4428,8.700000,6,1403.650000,1403.650000,43.833333,26.250000,0.0,0.0,Preventable Error
6750,8.216667,6,427.716667,427.716667,61.466667,26.633333,0.0,0.0,Preventable Error
9259,9.666667,6,682.433333,682.433333,44.033333,18.216667,0.0,0.0,Preventable Error
11108,10.683333,6,522.416667,522.416667,63.316667,33.233333,0.0,0.0,Preventable Error
11699,8.000000,6,1083.550000,1083.550000,44.116667,8.000000,0.0,0.0,Preventable Error
23769,8.133333,6,1597.500000,1597.500000,48.933333,16.283333,0.0,0.0,Preventable Error
24505,8.350000,6,344.233333,344.233333,50.016667,25.966667,0.0,0.0,Preventable Error


In [145]:
error_sample = final_output_df[final_output_df['Class'] == 'Preventable Error']
non_error_sample = final_output_df[final_output_df['Class'] == 'No Preventable Error'].head(2000)

In [146]:
final_output_df[final_output_df['Class'] == 'No Preventable Error']

,DURATION,Shift Hour Start,TotalWorkHoursToDate,RestaurantWorkHoursToDate,TotalWorkHoursInPrevious7Days,TotalWorkHoursInPrevious3Days,EmployeeErrorsToDateLag1,RestaurantTotalErrorsToDateLag1,Class
14,11.000000,6,22.000000,22.000000,11.000000,11.000000,0.0,0.0,No Preventable Error
17,8.066667,6,50.066667,50.066667,8.066667,8.066667,0.0,0.0,No Preventable Error
18,6.016667,6,56.083333,56.083333,6.016667,6.016667,0.0,0.0,No Preventable Error
20,8.033333,6,72.166667,72.166667,8.033333,8.033333,0.0,0.0,No Preventable Error
21,8.000000,6,80.166667,80.166667,8.000000,8.000000,0.0,0.0,No Preventable Error
...,...,...,...,...,...,...,...,...,...
66172,8.550000,15,304.950000,304.950000,42.783333,25.616667,0.0,0.0,No Preventable Error
66175,5.916667,15,328.016667,328.016667,23.066667,23.066667,0.0,0.0,No Preventable Error
66295,11.533333,4,31.350000,31.350000,21.266667,21.266667,0.0,0.0,No Preventable Error
66317,11.700000,4,33.983333,33.983333,22.916667,22.916667,0.0,0.0,No Preventable Error


In [147]:
model_sample = pd.concat([error_sample, non_error_sample])
model_sample 

,DURATION,Shift Hour Start,TotalWorkHoursToDate,RestaurantWorkHoursToDate,TotalWorkHoursInPrevious7Days,TotalWorkHoursInPrevious3Days,EmployeeErrorsToDateLag1,RestaurantTotalErrorsToDateLag1,Class
1496,8.583333,6,381.850000,381.850000,55.383333,25.700000,0.0,0.0,Preventable Error
1873,8.650000,6,865.050000,865.050000,51.816667,8.650000,0.0,0.0,Preventable Error
3728,8.633333,6,1399.200000,1399.200000,52.033333,25.883333,0.0,0.0,Preventable Error
4428,8.700000,6,1403.650000,1403.650000,43.833333,26.250000,0.0,0.0,Preventable Error
6750,8.216667,6,427.716667,427.716667,61.466667,26.633333,0.0,0.0,Preventable Error
...,...,...,...,...,...,...,...,...,...
6842,7.966667,6,1160.666667,1160.666667,43.383333,23.933333,1.0,1.0,No Preventable Error
6843,7.783333,6,1168.450000,1168.450000,51.166667,23.416667,1.0,1.0,No Preventable Error
6844,7.650000,6,1176.100000,1176.100000,43.466667,31.066667,1.0,1.0,No Preventable Error
6845,7.983333,6,1184.083333,1184.083333,39.050000,15.633333,1.0,1.0,No Preventable Error


In [148]:
# Convert 'Class' to binary where 'Preventable Error' = 1 and 'No Preventable Error' = 0
final_output_df['Class'] = np.where(final_output_df['Class'] == 'Preventable Error', 1, 0)

In [149]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Assuming 'Class' column is binary
X = final_output_df.drop('Class', axis=1)
y = final_output_df['Class']

# Splitting data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Applying SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

# Training a logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_res, y_res)

# Predicting and evaluating the model
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       1.00      0.74      0.85      6464
           1       0.01      0.89      0.02        18

    accuracy                           0.74      6482
   macro avg       0.50      0.81      0.43      6482
weighted avg       1.00      0.74      0.84      6482


In [150]:
# Convert X_test to DataFrame
X_test_df = pd.DataFrame(X_test, columns=X.columns)

# Add y_test and predictions to this DataFrame
X_test_df['y_test'] = y_test.values
X_test_df['predictions'] = predictions

# Subset where predictions are 1
predictions_1_df = X_test_df[X_test_df['predictions'] == 1]

# Count where y_test also equals 1
correct_predictions = predictions_1_df[predictions_1_df['y_test'] == 1].shape[0]

# Count where y_test does not equal 1
incorrect_predictions = predictions_1_df[predictions_1_df['y_test'] != 1].shape[0]

# Total predictions of 1
total_predictions_1 = predictions_1_df.shape[0]

# Percentage where y_test also equals 1
percentage_correct = (correct_predictions / total_predictions_1) * 100 if total_predictions_1 > 0 else 0

# Percentage where y_test doesn't equal 1
percentage_incorrect = (incorrect_predictions / total_predictions_1) * 100 if total_predictions_1 > 0 else 0

# Print results
print(f"Total 'predictions == 1': {total_predictions_1}")
print(f"Correct predictions (y_test == 1): {correct_predictions} ({percentage_correct:.2f}%)")
print(f"Incorrect predictions (y_test != 1): {incorrect_predictions} ({percentage_incorrect:.2f}%)")

Total 'predictions == 1': 1728
Correct predictions (y_test == 1): 16 (0.93%)
Incorrect predictions (y_test != 1): 1712 (99.07%)


In [151]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb  # Import XGBoost

# Assuming final_df is preloaded and 'Class' column is binary
X = final_output_df.drop('Class', axis=1)
y = final_output_df['Class']

# Splitting data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Applying SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

# Training an XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')  # Configure to avoid warnings
model.fit(X_res, y_res)

# Predicting and evaluating the model
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      6464
           1       0.03      0.17      0.04        18

    accuracy                           0.98      6482
   macro avg       0.51      0.57      0.52      6482
weighted avg       0.99      0.98      0.99      6482


In [152]:
# Convert X_test to DataFrame
X_test_df = pd.DataFrame(X_test, columns=X.columns)

# Add y_test and predictions to this DataFrame
X_test_df['y_test'] = y_test.values
X_test_df['predictions'] = predictions

# Subset where predictions are 1
predictions_1_df = X_test_df[X_test_df['predictions'] == 1]

# Count where y_test also equals 1
correct_predictions = predictions_1_df[predictions_1_df['y_test'] == 1].shape[0]

# Count where y_test does not equal 1
incorrect_predictions = predictions_1_df[predictions_1_df['y_test'] != 1].shape[0]

# Total predictions of 1
total_predictions_1 = predictions_1_df.shape[0]

# Percentage where y_test also equals 1
percentage_correct = (correct_predictions / total_predictions_1) * 100 if total_predictions_1 > 0 else 0

# Percentage where y_test doesn't equal 1
percentage_incorrect = (incorrect_predictions / total_predictions_1) * 100 if total_predictions_1 > 0 else 0

# Print results
print(f"Total 'predictions == 1': {total_predictions_1}")
print(f"Correct predictions (y_test == 1): {correct_predictions} ({percentage_correct:.2f}%)")
print(f"Incorrect predictions (y_test != 1): {incorrect_predictions} ({percentage_incorrect:.2f}%)")

Total 'predictions == 1': 119
Correct predictions (y_test == 1): 3 (2.52%)
Incorrect predictions (y_test != 1): 116 (97.48%)


# 5. Key Findings

## Raw Findings

## Summary of Findings from Hypothesis Testing and Logistic Regression Analysis